<a href="https://colab.research.google.com/github/alephpt/AI_Projects/blob/main/GPT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as fn
import requests

In [3]:
batch_size = 16 # Number of Sequences to Process in Parallel
block_size = 32 # Max Content Length for Predictions
max_iterations = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iterations = 200 # Number of iterations before saving state
n_embed = 64 # Maps the input sequence into high-dimensional space
n_head = 4 # Attention head for multi-head attention layers
n_layers = 4
dropout = 0.0

In [57]:
torch.manual_seed(1337)
bible_url = 'https://www.o-bible.com/download/kjv.txt'
bible = requests.get(bible_url)
bible_text = bible.text.split("\n")
bible_text.remove(bible_text[0])

In [58]:
print(bible_text[0])

Ge1:1 In the beginning God created the heaven and the earth.


In [59]:
vocab = sorted(list(set(bible.text)))
vocab_size = len(vocab)
print("Vocab Size:", vocab_size)

Vocab Size: 73


In [66]:
stoi = { ch: i for i, ch in enumerate(vocab)}
itos = { i: ch for i, ch in enumerate(vocab)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

nums = encode("Today is the day.")
words = decode(nums)

print("nums", nums)
print("words", words)

nums [41, 61, 50, 47, 71, 1, 55, 65, 1, 66, 54, 51, 1, 50, 47, 71, 8]
words Today is the day.


In [71]:
data = torch.tensor(encode(bible.text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([4404443]) torch.int64
tensor([29, 61, 58, 71,  1, 23, 55, 48, 58, 51,  6,  1, 22, 67, 66, 54, 61, 64,
        55, 72, 51, 50,  1,  4, 32, 55, 60, 53,  1, 31, 47, 59, 51, 65,  5,  1,
        43, 51, 64, 65, 55, 61, 60,  6,  1, 41, 51, 70, 66, 52, 55, 58, 51,  1,
        18, 12,  9, 10,  9, 14,  8,  0, 28, 51, 10, 19, 10,  1, 30, 60,  1, 66,
        54, 51,  1, 48, 51, 53, 55, 60, 60, 55, 60, 53,  1, 28, 61, 50,  1, 49,
        64, 51, 47, 66, 51, 50,  1, 66, 54, 51])


In [73]:
n = int(0.9 * len(data)) # train on the first 90%
training_data = data[:n]
validation_data = data[n:]
block_size = 8
training_data[:block_size + 1]
x = training_data[:block_size]
y = training_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([29]) the target: 61
when input is tensor([29, 61]) the target: 58
when input is tensor([29, 61, 58]) the target: 71
when input is tensor([29, 61, 58, 71]) the target: 1
when input is tensor([29, 61, 58, 71,  1]) the target: 23
when input is tensor([29, 61, 58, 71,  1, 23]) the target: 55
when input is tensor([29, 61, 58, 71,  1, 23, 55]) the target: 48
when input is tensor([29, 61, 58, 71,  1, 23, 55, 48]) the target: 58


In [74]:
def get_batch(split):
  data = training_data if split == "train" else validation_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
  return x, y

In [76]:
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

inputs:
torch.Size([16, 8])
tensor([[66, 54, 51, 64, 51, 61, 52,  1],
        [65, 54, 47, 58, 58,  1, 60, 61],
        [ 1, 66, 54, 51, 51,  1, 66, 61],
        [ 1, 47, 58, 65, 61,  1, 69, 55],
        [67, 60, 66, 61,  1, 66, 54, 51],
        [52, 66, 71,  1, 58, 61, 61, 62],
        [ 1, 47, 58, 58,  6,  1, 51, 68],
        [66,  1, 57, 51, 51, 62,  1, 55],
        [47, 65,  1, 65, 61,  6,  1, 69],
        [61, 68, 51, 64,  1, 47, 58, 58],
        [55, 64,  1, 51, 53, 53, 65,  1],
        [50,  6,  1, 30,  1, 69, 55, 58],
        [59,  8,  0, 31, 61, 65, 54, 10],
        [51,  1, 33, 36, 39, 25,  6,  1],
        [66, 54, 51,  1, 62, 58, 47, 49],
        [ 1, 66, 54, 51,  1, 33, 36, 39]])
targets:
torch.Size([16, 8])
tensor([[54, 51, 64, 51, 61, 52,  1, 66],
        [54, 47, 58, 58,  1, 60, 61, 66],
        [66, 54, 51, 51,  1, 66, 61,  1],
        [47, 58, 65, 61,  1, 69, 55, 66],
        [60, 66, 61,  1, 66, 54, 51, 59],
        [66, 71,  1, 58, 61, 61, 62, 65],
        [47, 58, 5

In [77]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t + 1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [66] the target: 54
when input is [66, 54] the target: 51
when input is [66, 54, 51] the target: 64
when input is [66, 54, 51, 64] the target: 51
when input is [66, 54, 51, 64, 51] the target: 61
when input is [66, 54, 51, 64, 51, 61] the target: 52
when input is [66, 54, 51, 64, 51, 61, 52] the target: 1
when input is [66, 54, 51, 64, 51, 61, 52, 1] the target: 66
when input is [65] the target: 54
when input is [65, 54] the target: 47
when input is [65, 54, 47] the target: 58
when input is [65, 54, 47, 58] the target: 58
when input is [65, 54, 47, 58, 58] the target: 1
when input is [65, 54, 47, 58, 58, 1] the target: 60
when input is [65, 54, 47, 58, 58, 1, 60] the target: 61
when input is [65, 54, 47, 58, 58, 1, 60, 61] the target: 66
when input is [1] the target: 66
when input is [1, 66] the target: 54
when input is [1, 66, 54] the target: 51
when input is [1, 66, 54, 51] the target: 51
when input is [1, 66, 54, 51, 51] the target: 1
when input is [1, 66, 54, 51, 51, 

In [ ]:
@torch 